# Calculus and Differential Equations

## Polynomials and calculus

A [polynomial](https://mathworld.wolfram.com/Polynomial.html) is a mathematical expression involving a sum of powers in one or more variables multiplied by coefficients. A polynomial in one variable (i.e., a univariate polynomial) with constant coefficients is given by

$$
a_n \, x^n + a_{n-1} \, x^{n-1} + \cdots + a_2 \, x^2 + a_1 \, x + a_0
$$

Calculus concerns the *differentiation* and *integration* of functions. Geometrically,

- the *derivative*, obtained by *differentiating*, of a function is its *gradient*
- the *integral*, obtained by *integrating*, of a function is the area that lies between the curve of the function and the $x$ axis, accounting for whether the curve lies above or below the axis

Differentiation: for powers of a variable $x$, the rule for differentiation is to multiply by the power and reduce the power by 1, so that

$$
\frac{\mathrm{d}(x^n)}{\mathrm{d}x} = n \, x^{n-1}
$$

Integration: for powers of a variable $x$, the rule for integration is to increase the power by 1 and divide by the new power, so that

$$
\int x^n \, \mathrm{d}x = \frac{x^{n+1}}{n+1}
$$